<a href="https://colab.research.google.com/github/youkiti/ARE/blob/postspinal-hypotension/fullText_processing_for_github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#driveをマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import glob
path = "WORKINGFOLDER"
os.chdir(path)

pdf_files = glob.glob("*.pdf")

In [ ]:
len(pdf_files)

308

In [ ]:
# pdf_files から ID_ と.pdf を""にリプレイスした値を作る
pdf_ids = [file.replace("ID_", "").replace(".pdf", "") for file in pdf_files]



In [ ]:
pdf_folders = [f"ID_{pdf_id}" for pdf_id in pdf_ids]

# pdfをAPIで処理

In [ ]:
!pip install -q pdfservices-sdk


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.4/274.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 894.6/894.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 12.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


In [ ]:
#セットアップ

import logging
import os
from datetime import datetime

from adobe.pdfservices.operation.auth.service_principal_credentials import ServicePrincipalCredentials
from adobe.pdfservices.operation.exception.exceptions import ServiceApiException, ServiceUsageException, SdkException
from adobe.pdfservices.operation.io.cloud_asset import CloudAsset
from adobe.pdfservices.operation.io.stream_asset import StreamAsset
from adobe.pdfservices.operation.pdf_services import PDFServices
from adobe.pdfservices.operation.pdf_services_media_type import PDFServicesMediaType
from adobe.pdfservices.operation.pdfjobs.jobs.extract_pdf_job import ExtractPDFJob
from adobe.pdfservices.operation.pdfjobs.params.extract_pdf.extract_element_type import ExtractElementType
from adobe.pdfservices.operation.pdfjobs.params.extract_pdf.extract_pdf_params import ExtractPDFParams
from adobe.pdfservices.operation.pdfjobs.params.extract_pdf.extract_renditions_element_type import \
    ExtractRenditionsElementType
from adobe.pdfservices.operation.pdfjobs.result.extract_pdf_result import ExtractPDFResult


import os.path
import json
from tqdm import tqdm
import shutil
from zipfile import ZipFile

from google.colab import userdata
PDF_SERVICES_CLIENT_ID = userdata.get('PDF_SERVICES_CLIENT_ID')
PDF_SERVICES_CLIENT_SECRET = userdata.get('PDF_SERVICES_CLIENT_SECRET')

os.environ["PDF_SERVICES_CLIENT_ID"] = PDF_SERVICES_CLIENT_ID
os.environ["PDF_SERVICES_CLIENT_SECRET"] = PDF_SERVICES_CLIENT_SECRET


# Initial setup, create credentials instance
credentials = ServicePrincipalCredentials(
    client_id=os.getenv('PDF_SERVICES_CLIENT_ID'),
    client_secret=os.getenv('PDF_SERVICES_CLIENT_SECRET'))

# Creates a PDF Services instance
pdf_services = PDFServices(credentials=credentials)


In [ ]:
import os
from tqdm import tqdm
from datetime import datetime

failed_files = []

for pdf_file in tqdm(pdf_files[5:], desc="Processing PDFs"):
    try:
        file = open(pdf_file, 'rb')
        input_stream = file.read()
        file.close()

        # Creates an asset(s) from source file(s) and upload
        input_asset = pdf_services.upload(input_stream=input_stream, mime_type=PDFServicesMediaType.PDF)

        # Create parameters for the job
        extract_pdf_params = ExtractPDFParams(
            elements_to_extract=[ExtractElementType.TEXT, ExtractElementType.TABLES],
            elements_to_extract_renditions=[ExtractRenditionsElementType.TABLES, ExtractRenditionsElementType.FIGURES],
        )

        # Creates a new job instance
        extract_pdf_job = ExtractPDFJob(input_asset=input_asset, extract_pdf_params=extract_pdf_params)

        # Submit the job and gets the job result
        location = pdf_services.submit(extract_pdf_job)
        pdf_services_response = pdf_services.get_job_result(location, ExtractPDFResult)

        # Get content from the resulting asset(s)
        result_asset: CloudAsset = pdf_services_response.get_result().get_resource()
        stream_asset: StreamAsset = pdf_services.get_content(result_asset)

        # Creates an output stream and copy stream asset's content to it
        output_file_path = 'extractTextTableInfoWithStylingFromPDF.zip'
        with open(output_file_path, "wb") as file:
            file.write(stream_asset.get_input_stream())

        # PDFファイル名（拡張子なし）を取得
        pdf_name = os.path.splitext(os.path.basename(pdf_file))[0]

        output_dir = os.path.join(os.path.dirname(pdf_file), pdf_name)

        # Create a directory with the same name as the PDF file (without extension) in the specified output path
        # 出力ディレクトリが存在しない場合は作成
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        # Extract the contents of the zip file to the created directory
        with ZipFile(output_file_path, 'r') as zip_ref:
            zip_ref.extractall(output_dir)

        # Remove the temporary zip file
        os.remove(output_file_path)

    except Exception as e:
        print(f"Error processing file: {pdf_file}")
        print(f"Error message: {str(e)}")
        failed_files.append(pdf_file)

# 現在の日時を取得
now = datetime.now()

# 日時を文字列に変換 (YYYY-MM-DD_HH-MM形式)
datetime_string = now.strftime("%Y-%m-%d_%H-%M")

# ファイル名の文字列を作成
filename = f"{datetime_string}_log.txt"

# Save the list of failed files to a log file in the current folder
with open(filename, "w") as log_file:
    for file_path in failed_files:
        log_file.write(file_path + "\n")

Processing PDFs:  22%|██▏       | 66/303 [13:33<38:20,  9.71s/it]

Error processing file: ID_34.pdf
Error message: description =DISQUALIFIED - File not suitable for content extraction: File contents are too complex for content extraction; requestTrackingId=7c622d9b-26f1-485c-898b-719cb231c8ac; statusCode=400; errorCode=DISQUALIFIED_COMPLEX_FILE


Processing PDFs:  24%|██▍       | 74/303 [14:51<39:56, 10.47s/it]

Error processing file: ID_43.pdf
Error message: description =DISQUALIFIED - File not suitable for content extraction: File contents are too complex for content extraction; requestTrackingId=82a3ceb3-fbff-4bd0-8b26-840bedd72703; statusCode=400; errorCode=DISQUALIFIED_COMPLEX_FILE


Processing PDFs: 100%|██████████| 303/303 [51:23<00:00, 10.18s/it]


In [ ]:

import pandas as pd

#json to text
def extract_combined_text(json_file_path):
    """
    Extracts and combines text from the structuredData.json file located at the given path.

    Args:
    json_file_path (str): The path to the structuredData.json file.

    Returns:
    str: The combined text extracted from the JSON file.
    """
    # Load the JSON data
    with open(json_file_path, 'r') as file:
        data = json.load(file)

    # Function to extract text from the JSON elements
    def extract_text(elements):
        texts = []
        for element in elements:
            if 'Text' in element:
                texts.append(element['Text'])
        return texts

    # Extract text from the elements
    text_contents = []
    if 'elements' in data:
        text_contents = extract_text(data['elements'])

    # Combine the extracted text into a single string
    combined_text = "\n".join(text_contents)

    return combined_text

# /*.jsonのファイルを検索し、リストアップ
article_texts = []

for p in pdf_folders:
    json_file_path = os.path.join(f"{p}/structuredData.json")
    if os.path.exists(json_file_path):
        combined_text = extract_combined_text(json_file_path)
        #pとcombined_textをペアにしてdataframeにする
        article_texts.append((p, combined_text))


# Create a DataFrame from the data_pairs list
df = pd.DataFrame(article_texts, columns=['PDF_ID', 'Combined_Text'])



In [ ]:
df.head()

,PDF_ID,Combined_Text
0,ID_211,Prophylactic Fixed-Rate Phenylephrine Versus N...
1,ID_146,Received: 2 August 2022 Revised: 27 January 20...
2,ID_58,Review began 07/17/2023 Review ended 07/26/202...
3,ID_174,Archives of Gynecology and Obstetrics (2020) 3...
4,ID_221,Received: 27 September 2020 | Revised: 19 Janu...


In [ ]:
df2 = pd.DataFrame(pdf_ids, columns=['filenames'])
df2["PDF_ID"] = pdf_folders

In [ ]:
df2.shape

(308, 2)

In [ ]:
dfm = pd.merge(df, df2, on="PDF_ID",how="outer")

In [ ]:
dfm.to_pickle("extracted_fulltexts.pkl")

# ここからChatGPTで処理

In [ ]:
import pandas as pd
# Load the DataFrame from the pickle file
df = pd.read_pickle("extracted_fulltexts.pkl")

In [ ]:
import re
# 数値部分と文字列部分に分割するための関数
def split_numeric_and_string(value):
    match = re.match(r'(\d+)(.*)', value)
    if match:
        return int(match.group(1)), match.group(2)
    return float('inf'), value  # 数値部分がない場合、大きな数値として扱う

# filenamesを分割して新しい列を追加
df[['filename_numeric', 'filename_string']] = df['filenames'].apply(split_numeric_and_string).apply(pd.Series)

# 数値部分でソートし、その後文字列部分でソート
df = df.sort_values(by=['filename_numeric', 'filename_string'])

# ソート後に不要な列を削除
df = df.drop(columns=['filename_numeric', 'filename_string'])

In [ ]:
import os
import pandas as pd
import glob
from pathlib import Path
import math
from google.colab import files

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.8 MB/s eta 0:00:00


In [ ]:
import os
from openai import OpenAI
import json
from tqdm import tqdm
import time

#secretで設定したキー
from google.colab import userdata
API_KEY = userdata.get('opanai_API_key')
#渡す

os.environ['OPENAI_API_KEY'] = API_KEY

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get(API_KEY),
)

In [ ]:
def get_completion(system_prompt, prompt, tools):
    function_name = tools[0]["function"]["name"]
    chat_completion = client.chat.completions.create(
        messages=[
                {"role":"system",
                "content":system_prompt},
                {
                "role": "user",
                "content": prompt
                }],
            tools = tools,
            tool_choice =  {"type": "function", "function": {"name": function_name}},
            temperature = 0,
            model = "gpt-4o"
          )

    arguments_json_str = chat_completion.choices[0].message.tool_calls[0].function.arguments

    arguments_json = json.loads(arguments_json_str)

    return arguments_json

In [ ]:
system_prompt = """
You are conducting a systematic review.
Please determine if the user inputted manuscript should be included or excluded.
Additionally, extract the name and dosage of any drugs used for treatment and placebo.
If you exclude, please explain why and select the category.

# Inclusion Criteria:
1. **Study Design:**
   - Randomized Controlled Trials (RCTs) assessing the comparative efficacy of norepinephrine (NE) and phenylephrine (PE) for postspinal hypotension in patients undergoing cesarean section (CS).
   - Studies must include the use of continuous infusion or intravenous bolus administration of NE or PE.
   - Both published and unpublished articles, abstracts of conferences, and letters are eligible.
   - No language or country restrictions will be applied.

2. **Participants:**
   - Adult patients (18 years or older) undergoing cesarean section under spinal anesthesia or combined spinal–epidural anesthesia (CSEA).
   - Includes both elective and non-elective (urgent and emergency) cesarean sections.
   - Participants can have singleton or multiple pregnancies (e.g., twin or triplet pregnancies).

3. **Interventions:**
   -Studies must include "at least" one of the following interventions:
　　　Phenylephrine (PE) administered in any form (bolus or infusion).
　　　Norepinephrine (NE) administered in any form (bolus or infusion).
　 - Studies comparing different doses of Phenylephrine or Norepinephrine are also included.
   - Both therapeutic (administered after hypotension onset) and prophylactic (administered before hypotension onset) administrations are considered.
   - Example: Phenylephrine bolus vs. Phenylephrine infusion -> included
   - Example: NE bolus vs. NE infusion -> included
   - Example: Phenylephrine bolus vs. NE bolus -> included

# Exclusion Criteria:

1. **Study Design:**
   - Cluster-randomized trials, crossover trials, quasi-experimental studies, and quasi-randomized trials will be excluded.
   - Studies that do not follow a randomized controlled trial design will be excluded.

2. **Participants:**
   - Children (under 18 years old).
   - Individuals with known hypersensitivity or allergy to study medications (e.g., Lidocaine, Bupivacaine, Fentanyl, PE, or NE).
   - Cases involving cesarean sections performed under general anesthesia or epidural anesthesia only.
   - Patients with known fetal abnormalities.

3. **Interventions:**
   - Studies evaluating other vasopressors or interventions outside the specified doses and administration methods for PE and NE.

"""

In [ ]:
#二値を返すtoolsを定義
tools =  [
        {
            "type": "function",
            "function": {
                "name": "manuscript_inclusion_decision",
                "description": "Determine if a manuscript meets the inclusion or exclusion criteria for the systematic review",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "inclusion": {
                            "type": "boolean",
                            "description": "True if the manuscript meets the inclusion criteria AND does not meet the exclusion criteria."
                        },
                        "exclusion_reason": {
                            "type": "string",
                            "description": "Provide a detailed explanation if the manuscript is excluded, based on the specified criteria and the relevant sections of the manuscript."
                        },
                        "exclusion_category": {
                            "type": "string",
                            "description": "Provide the exclusion reason category if the manuscript is excluded. The exclusion reason category should be one of the following: Wrong study design, Wrong population, Wrong drug, Others."
                        },
                         "intervention_or_control": {
                        "type": "array",
                        "items":{
                        "drug": {
                            "type": "string",
                            "description": "Names of drugs or placebo used for intervention or control."
                        },
                        "dosage": {
                            "type": "string",
                            "description": "Dosage of drugs or placebo used for intervention or control."
                        },
                        "administration_speed": {
                            "type": "string",
                            "description": "Administration speed used for intervention or control."
                        }
                        }
                        }
                    },
                    "required": ["inclusion","intervention_or_control"]
                }
            }
        }
    ]


In [ ]:
judgements = []

# Number of retries for failed API calls
max_retries = 3
retry_delay = 5  # seconds

for prompt in tqdm(df["Combined_Text"], desc="Processing Prompts"):
    for attempt in range(max_retries):
        try:
            request_payload = {"prompt": prompt, "tools": tools}
            # Call the get_completion function and extract the "judgement" from the result
            completion_result = get_completion(system_prompt, prompt, tools)
            # Append the judgement to the list
            judgements.append(completion_result)
            break  # Break out of the retry loop if successful
        except Exception as e:
            # Log the error or print it for debugging
            print(f"Error processing prompt (attempt {attempt+1}/{max_retries}): {prompt}")
            print(f"Error: {str(e)}")

            # Check if it's a specific JSON parsing error and handle it
            if "We could not parse the JSON body of your request" in str(e):
                print("Request payload:", json.dumps(request_payload, indent=2))  # Print the JSON payload
                if attempt < max_retries - 1:
                    print(f"Retrying in {retry_delay} seconds due to JSON parsing issue...")
                    time.sleep(retry_delay)
                else:
                    # Append a default value (e.g., None) to the list after max retries
                    judgements.append(None)
            else:
                # If it's a different error, append None and break the loop
                judgements.append(None)
                break

Processing Prompts:  11%|█         | 33/308 [04:56<26:51,  5.86s/it]

Error processing prompt (attempt 1/3): nan
Error: Error code: 400 - {'error': {'message': "We could not parse the JSON body of your request. (HINT: This likely means you aren't using your HTTP library correctly. The OpenAI API expects a JSON payload, but what was sent was not valid JSON. If you have trouble figuring out how to fix this, please contact us through our help center at help.openai.com.)", 'type': 'invalid_request_error', 'param': None, 'code': None}}
Request payload: {
  "prompt": NaN,
  "tools": [
    {
      "type": "function",
      "function": {
        "name": "manuscript_inclusion_decision",
        "description": "Determine if a manuscript meets the inclusion or exclusion criteria for the systematic review",
        "parameters": {
          "type": "object",
          "properties": {
            "inclusion": {
              "type": "boolean",
              "description": "True if the manuscript meets the inclusion criteria AND does not meet the exclusion criteria."


Processing Prompts:  11%|█         | 34/308 [05:06<32:52,  7.20s/it]

Error processing prompt (attempt 3/3): nan
Error: Error code: 400 - {'error': {'message': "We could not parse the JSON body of your request. (HINT: This likely means you aren't using your HTTP library correctly. The OpenAI API expects a JSON payload, but what was sent was not valid JSON. If you have trouble figuring out how to fix this, please contact us through our help center at help.openai.com.)", 'type': 'invalid_request_error', 'param': None, 'code': None}}
Request payload: {
  "prompt": NaN,
  "tools": [
    {
      "type": "function",
      "function": {
        "name": "manuscript_inclusion_decision",
        "description": "Determine if a manuscript meets the inclusion or exclusion criteria for the systematic review",
        "parameters": {
          "type": "object",
          "properties": {
            "inclusion": {
              "type": "boolean",
              "description": "True if the manuscript meets the inclusion criteria AND does not meet the exclusion criteria."


Processing Prompts:  14%|█▎        | 42/308 [06:07<29:16,  6.60s/it]

Error processing prompt (attempt 1/3): nan
Error: Error code: 400 - {'error': {'message': "We could not parse the JSON body of your request. (HINT: This likely means you aren't using your HTTP library correctly. The OpenAI API expects a JSON payload, but what was sent was not valid JSON. If you have trouble figuring out how to fix this, please contact us through our help center at help.openai.com.)", 'type': 'invalid_request_error', 'param': None, 'code': None}}
Request payload: {
  "prompt": NaN,
  "tools": [
    {
      "type": "function",
      "function": {
        "name": "manuscript_inclusion_decision",
        "description": "Determine if a manuscript meets the inclusion or exclusion criteria for the systematic review",
        "parameters": {
          "type": "object",
          "properties": {
            "inclusion": {
              "type": "boolean",
              "description": "True if the manuscript meets the inclusion criteria AND does not meet the exclusion criteria."


Processing Prompts:  14%|█▍        | 43/308 [06:17<34:01,  7.70s/it]

Error processing prompt (attempt 3/3): nan
Error: Error code: 400 - {'error': {'message': "We could not parse the JSON body of your request. (HINT: This likely means you aren't using your HTTP library correctly. The OpenAI API expects a JSON payload, but what was sent was not valid JSON. If you have trouble figuring out how to fix this, please contact us through our help center at help.openai.com.)", 'type': 'invalid_request_error', 'param': None, 'code': None}}
Request payload: {
  "prompt": NaN,
  "tools": [
    {
      "type": "function",
      "function": {
        "name": "manuscript_inclusion_decision",
        "description": "Determine if a manuscript meets the inclusion or exclusion criteria for the systematic review",
        "parameters": {
          "type": "object",
          "properties": {
            "inclusion": {
              "type": "boolean",
              "description": "True if the manuscript meets the inclusion criteria AND does not meet the exclusion criteria."


Processing Prompts: 100%|██████████| 308/308 [49:36<00:00,  9.66s/it]


In [ ]:
keys = set().union(*(d.keys() for d in judgements if d is not None))
# Create a None dictionary
none_dict = {key: None for key in keys}

# Replace None entries with the None dictionary
judgementsN = [j if j is not None else none_dict for j in judgements]

In [ ]:
judgements_df = pd.DataFrame(judgementsN)

In [ ]:
df_test = pd.concat([df, judgements_df], axis=1)

df_test.to_excel("0625judgementsFull.xlsx")